# Hybrid content based filter to reccomend classes to college students
## By: Khaled Hossain, Jahoon Koo, Carter Andrew, Jusung Kim
### 7/27/23
References: 
1. https://medium.com/grabngoinfo/recommendation-system-user-based-collaborative-filtering-a2e76e3e15c4
2. https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0284687
3. https://www.diva-portal.org/smash/get/diva2:1111865/FULLTEXT01.pdf
4. https://www.sciencedirect.com/science/article/pii/S1877050918302424
5. https://dl.acm.org/doi/10.1145/3460620.3460744
6. https://medium.com/analytics-vidhya/7-types-of-hybrid-recommendation-system-3e4f78266ad8

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

import sys

### Our datasets:
We pulled our data from https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0284687. This is a compiled list of course enrollments from Peking University between 2014 to 2021. The data sets comprise 4568 students, 5591 courses, and a total of 208949 course enrollments


In [3]:
student_data = pd.read_excel("dataset/StudentInformationTable.xlsx")
course_data = pd.read_excel("dataset/CourseInformationTable.xlsx")
career_data = pd.read_excel("dataset/CourseSelectionTable.xlsx")

In [ ]:
display(student_data.describe())
display(student_data.head(5))

In [ ]:
display(course_data.describe())
display(course_data.head(5))

In [ ]:
display(career_data.describe())
display(career_data.head(5))